Design a Python application to simulate a banking system involving two shared bank accounts, Account A (starting balance: $1000) and Account B (starting balance: $500). 
Write a function, transfer_money(sender_account, receiver_account, amount), and launch five threads that concurrently attempt to call this function with random amounts between $1 and $100.
1.	Implement Necessary Synchronization: Use two separate threading.Lock objects (one for each account) to protect the integrity of the individual account balances.
2.	Demonstrate Deadlock (Optional): Show how a deadlock can occur if the two locks are acquired in different orders by different threads (e.g., Thread 1 acquires Lock A then Lock B, while Thread 2 acquires Lock B then Lock A).
The final combined balance (A+B) must always equal the initial combined balance ($1500), regardless of the transfer order.

In [104]:
import threading
import random
import time

account_A = {"name": "Account A", "balance": 1000}
account_B = {"name": "Account B", "balance": 500}

lock_A = threading.Lock()
lock_B = threading.Lock()

def transfer_money(sender, receiver, amount):
    # Always lock accounts in a fixed order to prevent deadlock
    first_lock, second_lock = (lock_A, lock_B) if id(lock_A) < id(lock_B) else (lock_B, lock_A)
    with first_lock:
        with second_lock:
            if sender["balance"] >= amount:
                sender["balance"] -= amount
                receiver["balance"] += amount

def simulate_transfer():
    for _ in range(50):
        amount = random.randint(1, 100)
        if random.choice([True, False]):
            transfer_money(account_A, account_B, amount)
        else:
            transfer_money(account_B, account_A, amount)

threads = [threading.Thread(target=simulate_transfer, name=f"T{i+1}") for i in range(5)]

print("Starting Transfers...")
start = time.time()
for t in threads:
    t.start()
for t in threads:
    t.join()
end = time.time()

print("\n--- Final Results ---")
print(f"Account A Balance: ${account_A['balance']}")
print(f"Account B Balance: ${account_B['balance']}")
print(f"Total Combined: ${account_A['balance'] + account_B['balance']}")
print(f"Expected Total: $1500")
print(f"Time Taken: {end - start:.4f} sec")


Starting Transfers...

--- Final Results ---
Account A Balance: $947
Account B Balance: $553
Total Combined: $1500
Expected Total: $1500
Time Taken: 0.0023 sec


A company wants to analyze large text files. You are asked to design a Python program that:
Splits a given text file into five equal parts.
Creates five threads, where each thread:

a.	Reads one part of the file,

b.	Counts the number of words in its chunk,

c.	Stores the count in a shared list.

After all threads complete, the main thread should sum up the word counts and display the total number of words in the file.


In [105]:
import threading
import time

sample_text = ("Python multithreading is powerful and efficient. " * 1000).strip()
with open("sample_text.txt", "w") as f:
    f.write(sample_text)

word_counts = []
lock = threading.Lock()

def count_words_in_chunk(chunk):
    count = len(chunk.split())
    with lock:
        word_counts.append(count)

def threaded_word_count(filename, num_threads=5):
    with open(filename, "r") as f:
        text = f.read()
    size = len(text)
    chunk_size = size // num_threads
    threads = []
    start_time = time.time()
    for i in range(num_threads):
        start_idx = i * chunk_size
        end_idx = None if i == num_threads - 1 else (i + 1) * chunk_size
        chunk = text[start_idx:end_idx]
        t = threading.Thread(target=count_words_in_chunk, args=(chunk,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    total_words = sum(word_counts)
    print(f"Total word count: {total_words}")
    print(f"Processed in: {time.time() - start_time:.4f} seconds")

threaded_word_count("sample_text.txt")


Total word count: 6003
Processed in: 0.0008 seconds


Design a Python program to perform matrix multiplication using multithreading. You are given two square matrices A and B of size N x N. Create N threads, where each thread computes one row of the result matrix C = A × B. Use the threading module to launch the threads and join them after completion. Store the results in a shared 2D list (matrix C). Ensure synchronization so that no race condition occurs when multiple threads update the shared matrix. Compare the performance of the threaded version with a sequential version for large matrices.

In [106]:
import threading
import random
import time

N = 4
A = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
B = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]
C = [[0 for _ in range(N)] for _ in range(N)]

lock = threading.Lock()

def multiply_row(row_idx):
    global C
    for j in range(N):
        value = sum(A[row_idx][k] * B[k][j] for k in range(N))
        with lock:
            C[row_idx][j] = value

def threaded_matrix_multiply():
    threads = []
    start = time.time()
    for i in range(N):
        t = threading.Thread(target=multiply_row, args=(i,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    end = time.time()
    print(f"Threaded multiplication took: {end - start:.6f} sec")

def sequential_matrix_multiply():
    start = time.time()
    result = [[sum(A[i][k] * B[k][j] for k in range(N)) for j in range(N)] for i in range(N)]
    end = time.time()
    print(f"Sequential multiplication took: {end - start:.6f} sec")
    return result

threaded_matrix_multiply()
seq_result = sequential_matrix_multiply()

print("\nMatrix A:", A)
print("Matrix B:", B)
print("\nResult (Threaded):", C)
print("Result (Sequential):", seq_result)


Threaded multiplication took: 0.000817 sec
Sequential multiplication took: 0.000041 sec

Matrix A: [[10, 2, 10, 10], [8, 9, 10, 7], [2, 4, 5, 5], [4, 9, 1, 10]]
Matrix B: [[6, 10, 4, 9], [2, 8, 8, 2], [6, 3, 5, 3], [10, 3, 10, 10]]

Result (Threaded): [[224, 176, 206, 224], [196, 203, 224, 190], [100, 82, 115, 91], [148, 145, 193, 157]]
Result (Sequential): [[224, 176, 206, 224], [196, 203, 224, 190], [100, 82, 115, 91], [148, 145, 193, 157]]
